Originally run:

```bash
docker run -it --shm-size=8g  --gpus all \
-v"/shared/joregan/kaldi_swe/data:/opt/kaldi/egs/sprakbanken_swe/s5/data" \
-v"/shared/joregan/kaldi_swe/exp:/opt/kaldi/egs/sprakbanken_swe/s5/exp" \
-v"/shared/joregan/kaldi_swe/mfcc:/opt/kaldi/egs/sprakbanken_swe/s5/mfcc" kaldiasr/kaldi
```

(Failed) attempt with docker:

```slurm
#!/bin/bash
#SBATCH --job-name=kaldi_docker
#SBATCH --output=kaldi_docker_%j.out
#SBATCH --error=kaldi_docker_%j.err
#SBATCH --nodelist=deepspeech
#SBATCH --gres=gpu:1
#SBATCH --cpus-per-task=16
#SBATCH --mem=1M                    # workaround due to misreported memory
#SBATCH --time=2-00:00:00           # 2 days max runtime

echo "Starting job on $SLURMD_NODENAME at $(date)"
echo "Launching Kaldi Docker container..."

docker run --shm-size=8g --gpus all \
  -v "/shared/joregan/kaldi_swe/data:/opt/kaldi/egs/sprakbanken_swe/s5/data" \
  -v "/shared/joregan/kaldi_swe/exp:/opt/kaldi/egs/sprakbanken_swe/s5/exp" \
  -v "/shared/joregan/kaldi_swe/mfcc:/opt/kaldi/egs/sprakbanken_swe/s5/mfcc" \
  kaldiasr/kaldi bash -c "cd /opt/kaldi/egs/sprakbanken_swe/s5 && ./run.sh"

echo "Job completed at $(date)"
```